### Optimization


#### Initialization

In [3]:
import numpy as np
import random
import math
import pandas as pd
import matplotlib.pyplot as plt

#### Function

In [4]:
def simulation_model(parameter_ROP, parameter_EOQ):
    D = 1020
    C = 10
    N = 120
    F = 30
    P = 150
    z = 1.64
    LT_N = 3
    LT_F = 60
    K = 10
    h = 0.44
    delay = 7
    limit = 7
    considered = 10
    t1 = 0.03
    t2 = 0.06
    t3 = 0.1
    payment_term_soft = 5
    payment_term_norm = 1
    CAP = np.zeros((D+LT_F+delay,P))      # Inventory level  
    CAP[0,:] += 20
    pipeline = np.zeros((D+LT_F+delay,P))   # Inventory in the pipeline (from customer to warehouse)
    arrivals = np.zeros((D+LT_F+delay,P))   # Inventory in the pipeline (from supplier to warehouse)
    day_tot_cost = 0                                                   # Cost of sold goods per day
    account_payables = np.zeros(D+payment_term_soft)                   # Table of due payments
    day_tot_rev = 0
    revenues = np.zeros(D+payment_term_soft)                           # Table of cash inflows
    NOOS_price = np.random.uniform(90,180, N)          # NOOS products prices
    cost_N = np.random.uniform(30,80, N)               # NOOS products total cost
    ord_cost_N = cost_N.copy()                         # NOOS order cost
    cost_N[:3] += t1 * cost_N[:3]
    cost_N[3:7] += t2 * cost_N[3:7]
    cost_N[7:10] += t3 * cost_N[7:10]
    FASHION_price = np.random.uniform(160,210, F)      # FASHION products prices
    cost_F = np.random.uniform(80,160, F)              # FASHION products total cost
    ord_cost_F = cost_F.copy()                         # FASHION order cost
    cost_F[0:2] += t1 * cost_F[:2]
    cost_F[2:4] += t2 * cost_F[2:4]
    cost_F[4:5] += t3 * cost_F[4:5]
    prob = np.array([0.03, 0.05, 0.07, 0.1, 0.15, 0.2, 0.4])    # Purchase probabilities
    day_profit_N = 0                                                 # Daily NOOS profit
    month_N = np.zeros(D)                                            # List of daily NOOS profits
    profits_per_item_N = np.zeros((N), dtype=float)                  # Profits per NOOS items
    daily_soldq_N = np.zeros((D,N))                                  # List of daily sold NOOS items
    avg_q_N = np.zeros((D,N))                                                    # average qt sold per day per product - NOOS
    sigma_q_N = np.zeros((D,N))                                                  # std dev of qt sold per day per product - NOOS
    ROP_N = np.zeros((D,N))                                                      # Reorder point for NOOS
    EOQs_N = np.zeros((D,N))                                                     # Reorder quantity fo NOOS
    N_prof = np.zeros(D)
    day_profit_F = 0                                                 # Daily FASHION profit
    month_F = np.zeros(D)                                            # List of daily FASHION profit
    profits_per_item_F = np.zeros((F), dtype=float)                  # Profits per FASHION items
    daily_soldq_F = np.zeros((D,F))                                  # List of daily sold FASHION items
    avg_q_F = np.zeros((D,F))                                                    # average qt sold per day per product - FASHION
    sigma_q_F = np.zeros((D,F))                                                  # std dev of qt sold per day per product - FASHION
    ROP_F = np.zeros((D,F))                                                      # Reorder point for FASHION
    EOQs_F = np.zeros((D,F))                                                     # Reorder quantity fo FASHION
    F_prof = np.zeros(D)
    volume_N = 0                                                      # Total volume sold 
    volume_F = 0
    lost_sales = 0                                            # Lost sales (not spent residual budget) because out of stock
    lost_sales_euro = 0
    served_1 = 0                                              # How many times we served the customer with the most favourite item
    served_2 = 0
    served_3 = 0 
    served_4 = 0
    nw_served = 0
    not_served = 0
    purchased_items = np.zeros((D,C))                          # How many items does each customer buy 
    budg_track = np.zeros((D,C))                               # Keeping track of the budgets each customer has available
    
    for ix_d in range(D):
        preferences = np.random.rand(P, C) 
        budgets = np.random.uniform(250, 1000, C)
        day_profit_N = 0
        day_profit_F = 0
        day_tot_cost = 0
        day_tot_rev = 0
        #print("Day",ix_d)
        for ix_c in range(C):
            box = 0 
            fake_bdg = np.copy(budgets[ix_c])
            budg_track[ix_d,ix_c] += budgets[ix_c]
            ranked = np.argsort(preferences[:,ix_c])
            ranked_preferences = ranked[::-1]  
            PLS = np.zeros(considered)                                                                   # Potential Lost sales
            rk = 0                                                                                       # Put in order PLS
            service_check = np.zeros(limit)
            sc = 0
            #print("Budget customer",ix_c+1, ": ", budgets[ix_c])
            for ix_rp in ranked_preferences[0:considered]:
                if ix_rp < N:
                    #print("  ", str(ix_rp) + ".", preferences[ix_rp,ix_c], NOOS_price[ix_rp], budgets[ix_c])
                    if CAP[ix_d,ix_rp] > 0 and box < limit:   
                        service_check[sc] += ix_rp
                        sc += 1
                        dec = np.random.choice([0,1] , 1, p=[prob[box], 1-prob[box]])
                        box += 1
                        if dec == 1 and NOOS_price[ix_rp] < budgets[ix_c]:
                            budgets[ix_c] -= NOOS_price[ix_rp]   
                            fake_bdg -= NOOS_price[ix_rp] 
                            #print(" ✓✓")
                            volume_N += 1
                            day_tot_rev += NOOS_price[ix_rp]
                            day_profit_N += (NOOS_price[ix_rp] - cost_N[ix_rp])
                            N_prof[ix_d] += (NOOS_price[ix_rp] - cost_N[ix_rp])
                            profits_per_item_N[ix_rp] +=  NOOS_price[ix_rp] - cost_N[ix_rp]
                            day_tot_cost += cost_N[ix_rp] 
                            daily_soldq_N[ix_d,ix_rp] += 1
                            CAP[ix_d,ix_rp] -= 1
                            purchased_items[ix_d,ix_c] += 1
                        else:                                              
                            CAP[ix_d,ix_rp] -= 1
                            pipeline[ix_d,ix_rp] += 1
                            #print("xx")
                    elif CAP[ix_d,ix_rp] == 0 and box <= limit and NOOS_price[ix_rp] < fake_bdg:  
                        PLS[rk] += NOOS_price[ix_rp]
                        rk += 1
                        #print("**")
                    avg_q_N[ix_d,ix_rp] = np.mean(daily_soldq_N[:ix_d,ix_rp])
                    sigma_q_N[ix_d,ix_rp] = np.std(daily_soldq_N[:ix_d,ix_rp])
                    if ix_d == 0:
                        ROP_N[ix_d,ix_rp] = (LT_N * avg_q_N[ix_d,ix_rp]) + (z * sigma_q_N[ix_d,ix_rp] * math.sqrt(LT_N)) * parameter_ROP
                        EOQs_N[ix_d,ix_rp] = math.sqrt((2 * K * avg_q_N[ix_d,ix_rp]) / h) * parameter_EOQ
                    else:
                        ROP_N[ix_d,ix_rp] = int((LT_N * avg_q_N[ix_d,ix_rp]) + (z * sigma_q_N[ix_d,ix_rp] * math.sqrt(LT_N))) * parameter_ROP
                        EOQs_N[ix_d,ix_rp] = int(math.sqrt((2 * K * avg_q_N[ix_d,ix_rp]) / h)) * parameter_EOQ         
                else:
                    ixF = ix_rp-N
                    #print("  ", str(ix_rp) + ".", preferences[ix_rp,ix_c], FASHION_price[ixF], budgets[ix_c])    
                    if CAP[ix_d,ix_rp] > 0 and box < limit:
                        service_check[sc] += ix_rp
                        sc += 1
                        dec = np.random.choice([0,1] , 1, p=[prob[box], 1-prob[box]])
                        box += 1      
                        if dec == 1 and FASHION_price[ixF] < budgets[ix_c]:
                            budgets[ix_c] -= FASHION_price[ixF]   
                            fake_bdg -= FASHION_price[ixF] 
                            #print(" ✓✓")
                            volume_F += 1
                            day_tot_rev += FASHION_price[ixF]
                            day_profit_F += (FASHION_price[ixF] - cost_F[ixF])
                            F_prof[ix_d] += (FASHION_price[ixF] - cost_F[ixF])
                            profits_per_item_F[ixF] +=  FASHION_price[ixF] - cost_F[ixF]
                            day_tot_cost += cost_F[ixF] 
                            daily_soldq_F[ix_d,ixF] += 1
                            CAP[ix_d,ix_rp] -= 1
                            purchased_items[ix_d,ix_c] += 1
                        else:                                              
                            CAP[ix_d,ix_rp] -= 1
                            pipeline[ix_d,ix_rp] += 1
                            #print("xx")
                    elif CAP[ix_d,ix_rp] == 0 and box <= limit and FASHION_price[ixF] < fake_bdg:
                        PLS[rk] += FASHION_price[ixF]
                        rk += 1
                        #print("**")
                    avg_q_F[ix_d,ixF] = np.mean(daily_soldq_F[:ix_d,ixF])
                    sigma_q_F[ix_d,ixF] = np.std(daily_soldq_F[:ix_d,ixF])
                    if ix_d == 0:
                        ROP_F[ix_d,ixF] = (LT_F * avg_q_F[ix_d,ixF]) + (z * sigma_q_F[ix_d,ixF] * math.sqrt(LT_F)) * parameter_ROP
                        EOQs_F[ix_d,ixF] = math.sqrt((2 * K * avg_q_F[ix_d,ixF]) / h) * parameter_EOQ 
                    else:
                        ROP_F[ix_d,ixF] = int((LT_F * avg_q_F[ix_d,ixF]) + (z * sigma_q_F[ix_d,ixF] * math.sqrt(LT_F))) * parameter_ROP
                        EOQs_F[ix_d,ixF] = int(math.sqrt((2 * K * avg_q_F[ix_d,ixF]) / h)) * parameter_EOQ 
        
            for ix_sc in service_check[service_check!=0]:
                if ix_sc == ranked_preferences[0]:
                    served_1 += 1
                    break
                elif ix_sc == ranked_preferences[1]:
                    served_2 += 1
                    break
                elif ix_sc == ranked_preferences[2]:
                    served_3 += 1
                    break
                elif ix_sc == ranked_preferences[3]:
                    served_4 += 1
                    break
                else:
                    nw_served += 1
                    break

            if box == 0:
                not_served += 1

            for ix_bp in PLS[PLS!=0]:
                if ix_bp < fake_bdg:
                    lost_sales += 1
                    lost_sales_euro += ix_bp
                    fake_bdg -= ix_bp
                    
        for ix_pn in range(P):
            if ix_pn < N:
                if ix_d-delay >= 0:
                    CAP[ix_d,ix_pn] += pipeline[ix_d-delay,ix_pn]
                if ix_d-LT_N >= 0:
                    CAP[ix_d,ix_pn] += arrivals[ix_d-LT_N,ix_pn]
                CAP[ix_d+1,ix_pn] = CAP[ix_d,ix_pn] 
                if CAP[ix_d,ix_pn] + np.sum(pipeline[ix_d:ix_d+delay,ix_pn]) + np.sum(arrivals[ix_d:ix_d+LT_N,ix_pn]) <= ROP_N[ix_d,ix_pn]:
                    arrivals[ix_d+LT_N,ix_pn] += EOQs_N[ix_d,ix_pn]  
                    day_tot_cost += K
            else:
                ix_pf = ix_pn-N
                if ix_d-delay >= 0:
                    CAP[ix_d,ix_pn] += pipeline[ix_d-delay,ix_pn]
                if ix_d-LT_F >= 0:
                    CAP[ix_d,ix_pn] += arrivals[ix_d-LT_F,ix_pn]
                CAP[ix_d+1,ix_pn] = CAP[ix_d,ix_pn] 
                if CAP[ix_d,ix_pn] + np.sum(pipeline[ix_d:ix_d+delay,ix_pn]) + np.sum(arrivals[ix_d:ix_d+LT_F,ix_pn]) <= ROP_F[ix_d,ix_pf]:
                    arrivals[ix_d+LT_F,ix_pn] += EOQs_F[ix_d,ix_pf]
                    day_tot_cost += K
            day_tot_cost += h * CAP[ix_d,ix_pn]
    
        revenues[ix_d] += day_tot_rev
        account_payables[payment_term_soft + ix_d] += day_tot_cost * 0.5
        account_payables[payment_term_norm + ix_d] += day_tot_cost * 0.5
        #print("NOOS DAY PROFIT :", day_profit_N)
        month_N[ix_d] += day_profit_N 
        #print("FASHION DAY PROFIT :", day_profit_F)        
        month_F[ix_d] += day_profit_F
        
    month_profit_N = np.sum(month_N)                                             # NOOS monthly profit
    #print("NOOS TOTAL PROFIT :", month_profit_N)
    #print("PROFITS PER ITEM - NOOS: ",profits_per_item_N)
    ranked_N = np.argsort(profits_per_item_N)
    ranked_1 = ranked_N[::-1]                                                    # Sorted NOOS products by performance
    #print("TOP PERFORMING NOOS PRODUCTS: ", ranked_1)
    #print("Average Inventory Level - NOOS: ", np.nanmean(CAP[:D,:N]))
    #print("Average Reorder point - NOOS: ", np.nanmean(ROP_N[:D,:]))
    #print("Average Order quantity - NOOS", np.nanmean(EOQs_N[:D,:]))
    
    print("")
    
    month_profit_F = np.sum(month_F)                                           # FASHION monthly profit
    #print("FASHION TOTAL PROFIT :", month_profit_F)
    #print("PROFITS PER ITEM - FASHION: ",profits_per_item_F)
    ranked_F = np.argsort(profits_per_item_F)
    ranked_2 = ranked_F[::-1]                                                  # Sorted FASHION products by performance
    #print("TOP PERFORMING FASHION PRODUCTS: ", ranked_2)
    #print("Average Inventory Level - FASHION: ", np.nanmean(CAP[:D,N:P]))
    #print("Average Reorder point - FASHION: ", np.nanmean(ROP_F[:D,:]))
    #print("Average Order quantity - FASHION", np.nanmean(EOQs_F[:D,:]))
    
    print("")
    
    tot_lost_sales_ratio = (lost_sales / (volume_N + volume_F))
    print("LOST SALES RATIO: ", tot_lost_sales_ratio)
    service_quality_ratio = np.sum(served_1) / (C * D)
    print("SERVICE QUALITY: ", service_quality_ratio)
    sold_quantities_N = np.sum(daily_soldq_N, axis = 0)
    sold_quantities_F = np.sum(daily_soldq_F, axis = 0)
    #print("BEST SELLING PRODUCTS (sold units): ", sold_quantities_N, sold_quantities_F)
    #print("MOST PROFITABLE PRODUCTS: ", ranked_1, ranked_2)
    print("TOTAL REVENUE OF THE SYSTEM: ", np.sum(revenues))
    print("TOTAL COST OF THE SYSTEM: ", np.sum(account_payables))
    
    print("")
    
    #print("CHECKLIST OF DUE PAYMENTS", account_payables)
    #print("CASH INFLOWS", revenues)
    profits_allocation = revenues - account_payables
    #print("PROFIT ALLOCATION", profits_allocation)
    
    print("")
    
    plt.figure(figsize=(6, 3), layout='constrained')
    av_inv_lev = np.zeros(D)
    for inv in range(D):
        av_inv_lev[inv] += np.mean(CAP[inv,:N])
    plt.plot(av_inv_lev, markersize = 3, color='black', linewidth=0.2)
    plt.axhline(y=np.nanmean(ROP_N), linestyle='--', linewidth=0.9, color='g', label="ROP")
    plt.axhline(y=np.nanmean(EOQs_N), linestyle='--', linewidth=0.9, color="orange", label="EOQ")
    plt.axhline(y=np.nanmean(CAP[:,:N]), linestyle='--', linewidth=0.9, color="r", label="INV")
    plt.axhline(y=np.std(CAP[:,:N]), linestyle='--', linewidth=0.9, color="yellow", label="STD")
    plt.title("Average Inventory Level among NOOS Products")
    plt.xlabel("DAYS")
    plt.ylabel("Inventory level")
    plt.xticks(np.arange(0,D,100))
    #plt.yticks(np.arange(0,20,2))
    plt.style.use("fast")
    plt.legend(loc=1)
    #plt.savefig("plot1.png")
    plt.show()
    
    print("")
    
    plt.figure(figsize=(6, 4), layout='constrained')
    width = 0.25 
    m_indexes = np.arange(0,34)
    tot_prof = np.array(N_prof+F_prof)
    monthly_prof = np.zeros(34)
    monthly_rev = np.zeros(34)
    bg = 0
    end = 30
    for ix_prof in range(34):
        monthly_prof[ix_prof] += np.nanmean(tot_prof[bg:end])
        monthly_rev[ix_prof] += np.nanmean(revenues[bg:end])
        bg += 30
        end += 30
    av_m_prof = np.nanmean(monthly_prof)
    av_m_rev = np.nanmean(monthly_rev)
    plot1 = plt.bar(m_indexes + width, monthly_prof, width=width, color="#444444", label="profits")
    plot2 = plt.bar(m_indexes, monthly_rev, width=width, color="#e5ae38", label="revenues")
    plt.axhline(y=av_m_prof, linestyle='--', linewidth=0.7, color="#444444", label=np.round(av_m_prof,2))
    plt.axhline(y=av_m_rev, linestyle='--', linewidth=0.7, color="#e5ae38", label=np.round(av_m_rev))
    plt.legend()
    plt.title("profit per day")
    plt.xlabel("days")
    plt.ylabel("profits in €")
    plt.style.use("fast")
    plt.tight_layout()
    #plt.savefig("plot2.png")
    plt.show()
    
    print("")
    #np.sum(revenues) & lost_sales_€
    slices = np.array([np.sum(revenues), lost_sales_euro])
    labels = ["Actual Sales", "Lost Sales"]
    plt.figure(figsize=(6, 4), layout='constrained')
    explode = [0,0.12]
    plt.pie(slices, labels=labels, wedgeprops={"edgecolor":"black"}, explode=explode,
            shadow=True, autopct=lambda p: '{:.2f}%({:.0f})'.format(p,(p/100)*slices.sum()))
    plt.title("Potential Sales (€)")
    plt.style.use("fast")
    plt.tight_layout()
    plt.show()
    
    print("")
    
    others = served_3 + served_4 + nw_served + not_served
    pieces = np.array([served_1, served_2, others])
    tags = ["1st pref", "2nd pref", "others"]
    plt.figure(figsize=(6, 4), layout='constrained')
    explode = [0,0,0.3]
    plt.pie(pieces, labels=tags,colors=["#4CAF50","orange","coral"],
            wedgeprops={"edgecolor":"black"}, explode=explode, shadow=True, autopct="%1.1f%%")
    plt.title("Service Quality")
    plt.style.use("fast")
    plt.tight_layout()
    plt.show()
    data_service = {'Class':["1st pref", "2nd pref", "others"], 'Number of Customers':[served_1, served_2, others], 
                    'Percentages':[(served_1/(C*D)), (served_2/(C*D)), (others/(C*D))]}
    df = pd.DataFrame(data_service)
    df.style
    display(df)
    
    print("")
    
    x = (budg_track.flatten())
    y = (purchased_items.flatten())
    m, b = np.polyfit(x, y, 1)
    plt.scatter(x, y, s=1, c="y", marker=".", edgecolor="black", linewidth=3, alpha=0.9)
    plt.plot(x, m*x+b, linewidth=0.7, c="r")
    plt.xlabel("Available Budget")
    plt.ylabel("Number of purchased items")
    plt.yticks(np.arange(8))
    plt.title("Budget-Purchased items relationship")
    plt.style.use("fast")
    plt.grid(True)
    plt.show()
    return 

In [6]:
for hj in range(5):
    print(simulation_model(parameter_ROP=1, parameter_EOQ=1.05))